In [1]:
import pickle
from attention import *
from torch.utils.data import DataLoader
from pytorch_util import RAdam,trainable_parameter
from torch.optim.lr_scheduler import ReduceLROnPlateau
#from pytorch_transformers import AdamW,WarmupLinearSchedule

In [2]:
batch_size = 1024
dim = 256
head_d = 8
head = SimplyInteraction #SimplyHead
encoder_layer = 4
epochs = 10
clip = 0.1
logLoss = False
EncoderLayer=TransformerEncoderLayer_BN

dim_feedforward = 1024
dropout = 0.01
lr = 3e-2

In [3]:
with open('../Data/train_data_attention_node.pickle', 'rb') as handle:
    train_node = pickle.load(handle)
with open('../Data/train_data_attention_edge.pickle', 'rb') as handle:
    train_edge = pickle.load(handle)
with open('../Data/train_data_attention_edge_y.pickle', 'rb') as handle:
    train_y = pickle.load(handle)
with open('../Data/train_data_ind.pickle', 'rb') as handle:
    ind = pickle.load(handle)

In [4]:
train_dl = attentionDataset(train_node[:4000000],train_edge[:4000000],ind[:4000000],train_y[:4000000])
train_dl = DataLoader(train_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn,num_workers=4)
val_dl = attentionDataset(train_node[4000000:],train_edge[4000000:],ind[4000000:],train_y[4000000:])
val_dl = DataLoader(val_dl,shuffle=True,batch_size=batch_size,collate_fn=collate_fn,num_workers=4)

In [38]:
model=Attention2(dim,encoder_layer,head_d,head,EncoderLayer,
                 dropout=dropout,dim_feedforward=dim_feedforward).to('cuda')
paras = trainable_parameter(model)
opt = RAdam(paras,lr=lr,weight_decay=1e-2)
#opt = AdamW(paras, lr=lr, correct_bias=False)
scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5,min_lr=1e-04)
#scheduler = WarmupLinearSchedule(opt, warmup_steps=40, t_total=100)

In [6]:
# out,mask,edge,y = next(iter(train_dl))
# out,mask,edge,y = out.to('cuda:0'),mask.to('cuda:0'),edge.to('cuda:0'),y.to('cuda:0')

# opt.zero_grad()
# loss,loss_perType = model(out,mask,edge,y,logLoss)
# loss.backward()

# [(n,p.std()) for n,p in model.named_parameters()]

# [(n,p.grad.mean()) for n,p in model.named_parameters()]

In [39]:
model,bestOpt,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,\
                                       scheduler=scheduler,logLoss=logLoss)

epoch:0, train_loss: +5.183, val_loss: +1.235, 
train_vector: +2.73|+2.31|+0.97|+1.09|+0.97|+0.87|+1.06|-0.14, 
val_vector  : +3.25|+1.99|+0.87|+0.71|+0.93|+0.82|+1.15|-0.13

epoch:1, train_loss: +5.125, val_loss: +1.152, 
train_vector: +2.78|+2.27|+0.97|+0.90|+0.97|+0.89|+1.06|-0.15, 
val_vector  : +2.57|+2.04|+0.87|+0.71|+0.93|+0.82|+1.15|-0.14

epoch:2, train_loss: +4.854, val_loss: +1.174, 
train_vector: +2.61|+2.32|+0.97|+0.91|+0.97|+0.89|+1.06|-0.15, 
val_vector  : +2.92|+1.81|+0.87|+0.71|+0.93|+0.83|+1.15|-0.13

epoch:3, train_loss: +4.889, val_loss: +1.158, 
train_vector: +2.63|+2.29|+0.98|+0.90|+0.97|+0.89|+1.06|-0.14, 
val_vector  : +2.64|+1.99|+0.87|+0.71|+0.92|+0.83|+1.15|-0.12



KeyboardInterrupt: 